In [1]:
import matplotlib.pyplot as plt
import numpy as np
import copy
import h5py
import os
import seaborn as sns


In [245]:
# Wavefunction generation
from pyscf import lib, gto, scf
from pyscf import gto, mp, mcscf
def H2_casci(scf_checkfile, ci_checkfile):
    mol = gto.M(
        atom="H 0. 0. 0.0; H 0. 0. 1.4",
        basis=f"ccecpccpvdz",
        unit="bohr",
        charge=0,
        spin=0,
        verbose=1,
    )
    mf = scf.ROHF(mol).run()
    mf.chkfile = scf_checkfile
    mf.kernel()
    mc = mcscf.CASCI(mf, 6, 2)
    # mc.fcisolver.nroots = 4
    mc.kernel()
    print(mc.__dict__.keys())
    with h5py.File(ci_checkfile, "a") as f:
        f.create_group("ci")
        f["ci/ncas"] = mc.ncas
        f["ci/nelecas"] = list(mc.nelecas)
        f["ci/ci"] = mc.ci
        f["ci/mo_coeff"] = mc.mo_coeff    
    return mol, mf, mc

In [246]:
scf_checkfile = 'rohf.chk'
ci_checkfile = 'casci.chk'
for fname in [scf_checkfile, ci_checkfile]:
    if os.path.isfile(fname):
        os.remove(fname)
mol, mf, mc = H2_casci(scf_checkfile, ci_checkfile)

dict_keys(['mol', '_scf', 'verbose', 'stdout', 'max_memory', 'ncas', 'nelecas', '_ncore', 'fcisolver', 'frozen', 'extrasym', 'e_tot', 'e_cas', 'ci', 'mo_coeff', 'mo_energy', 'mo_occ', 'converged'])


In [247]:
i = 0
j = 1
ovlp = numpy.dot(mc.ci[i].ravel().conj(),mc.ci[j].ravel())
ovlp

9.758567103948153e-16

In [252]:
ci0 = copy.deepcopy(mc.ci[0])
ci1 = copy.deepcopy(mc.ci[0])
ci0[1]*=0
ci0[2]*=0
ci0[3]*=0
ci0[4]*=0
ci0[5]*=0

ci1[1]*=0
ci1[2]*=0
ci1[3]*=0
ci1[4]*=0
ci1[5]*=0
ovlp = numpy.dot(ci0.ravel().conj(),ci1.ravel())
ovlp

0.9838400648615034

In [254]:
myhf1 = gto.M(atom='H 0 0 0; F 0 0 1.1', basis='6-31g', verbose=0).apply(scf.RHF).run()
norb = 4
nelec = 4
mc1 = mcscf.CASCI(myhf1, norb, nelec).run()
mo0 = mc1.mo_coeff
ci0 = mc1.ci
ci1 = copy.deepcopy(mc1.ci)
ci0[0]*=0
ci0[2]*=0
ci0[3]*=0
ci0[4]*=0
ci0[5]*=0

ci1[0]*=0
ci1[2]*=0
ci1[3]*=0
ci1[4]*=0
ci1[5]*=0

print('CASCI energy of mol1', mc1.e_tot)

i = 5
j = i
s = numpy.einsum('pi,pq,qj->ij', mo0[:,i:i+norb], myhf1.get_ovlp(), mo0[:,j:j+norb])
# here s = 1
# s12 = pyscf.gto.intor_cross('int1e_ovlp', myhf1.mol, myhf1.mol)
# s12 is equal to myhf1.get_ovlp()

# fci.addons.overlap(ci0,ci1, norb, nelec, s)
# When s = 1, above is equal to ci0.ravel().conj()*ci1.ravel()
ovlp = numpy.dot(ci0.ravel().conj(),ci1.ravel())
ovlp

CASCI energy of mol1 -99.95988952535133


0.00018125783588489701

In [238]:
ci0

FCIvector([[ 9.99783973e-01, -8.32667268e-17,  1.13059448e-18,
            -1.40306585e-17,  4.63983384e-18,  3.56889192e-19],
           [ 0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
            -0.00000000e+00,  0.00000000e+00, -0.00000000e+00],
           [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
             0.00000000e+00, -0.00000000e+00,  0.00000000e+00],
           [-0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
            -0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
             0.00000000e+00, -0.00000000e+00,  0.00000000e+00],
           [ 0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
            -0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [255]:
myhf1 = pyscf.M(atom='H 0 0 0; F 0 0 1.1', basis='6-31g', verbose=0).RHF().run()
e1, ci1 = pyscf.fci.FCI(myhf1).kernel()
print('FCI energy of mol1', e1)

myhf2 = pyscf.M(atom='H 0 0 0; F 0 0 1.2', basis='6-31g', verbose=0).RHF().run()
e2, ci2 = pyscf.fci.FCI(myhf2).kernel()
print('FCI energy of mol2', e2)

FCI energy of mol1 -100.1021146566482
FCI energy of mol2 -100.08393757040976


In [256]:
s12 = pyscf.gto.intor_cross('int1e_ovlp', myhf1.mol, myhf2.mol)
s12 = reduce(numpy.dot, (myhf1.mo_coeff.T, s12, myhf2.mo_coeff))

In [257]:
norb = myhf2.mo_energy.size
nelec = myhf2.mol.nelectron
print('<FCI-mol1|FCI-mol2> = ', pyscf.fci.addons.overlap(ci1, ci2, norb, nelec, s12))

<FCI-mol1|FCI-mol2> =  0.39963236744209335


In [258]:
ci1

FCIvector([[ 9.77930458e-01, -8.66822956e-18, -4.80516072e-17, ...,
            -5.46783436e-20,  4.39942545e-08, -2.06192228e-07],
           [-8.66822956e-18, -1.03109302e-02, -3.33555068e-19, ...,
            -6.90896377e-07, -2.25751916e-22, -1.18573500e-22],
           [-4.80516072e-17,  6.07343277e-18, -1.03109302e-02, ...,
            -1.38800602e-12, -5.32659154e-23, -4.17753258e-23],
           ...,
           [-5.46783436e-20, -6.90896377e-07, -1.38800602e-12, ...,
             1.49703075e-07,  6.04115599e-24,  6.31239743e-24],
           [ 4.39942545e-08, -2.25751916e-22, -5.32659154e-23, ...,
             6.04115599e-24, -1.18210289e-08, -9.31122348e-09],
           [-2.06192228e-07, -1.18573500e-22, -4.17753258e-23, ...,
             6.31239743e-24, -9.31122348e-09, -5.37955222e-09]])

In [260]:
ci2

FCIvector([[ 9.73631787e-01, -3.29055833e-17, -2.72022635e-17, ...,
             9.20832073e-21,  1.33857331e-07, -1.71869554e-07],
           [-3.29055833e-17, -1.09924665e-02, -7.65279639e-18, ...,
            -6.24654459e-07,  4.15850600e-22,  3.44380213e-22],
           [-2.72022635e-17, -5.89268937e-18, -1.09924665e-02, ...,
            -3.24046528e-12, -4.14672828e-22, -8.11320060e-23],
           ...,
           [ 9.20832073e-21, -6.24654459e-07, -3.24046529e-12, ...,
             1.57243756e-07, -5.17748283e-23, -3.72015098e-24],
           [ 1.33857331e-07,  4.15850600e-22, -4.14672828e-22, ...,
            -5.17748283e-23, -1.76982575e-08, -1.00673075e-08],
           [-1.71869554e-07,  3.44380213e-22, -8.11320060e-23, ...,
            -3.72015098e-24, -1.00673075e-08, -3.98094148e-09]])

In [262]:
norb = 4
nelec = (2,2)
ua = numpy.linalg.svd(numpy.random.random((norb+1,norb+1)))[0]
ub = numpy.linalg.svd(numpy.random.random((norb+1,norb+1)))[0]
s = numpy.dot(ua[:,:norb].T, ub[:,:norb])

strs = fci.cistring.make_strings(range(norb), nelec[0])
na = len(strs)
ci0 = numpy.random.random((na,na))
ci0 /= numpy.linalg.norm(ci0)
ci1 = numpy.random.random((na,na))
ci1 /= numpy.linalg.norm(ci1)

ovlpa = numpy.zeros((na,na))
ovlpb = numpy.zeros((na,na))

In [268]:
for ia in range(na):
    for ja in range(na):
        ovlpa[ia,ja] = fci.addons.det_overlap(strs[ia], strs[ja], norb, s)
for ib in range(na):
    for jb in range(na):
        ovlpb[ib,jb] = fci.addons.det_overlap(strs[ib], strs[jb], norb, s)

In [271]:
ovlp = numpy.einsum('ab,ij,ai,bj->', ci0, ci1, ovlpa, ovlpb)
ovlp

0.00880458950546415

In [272]:
ref = fci.addons.overlap(ci0, ci1, norb, nelec, s)
ref

0.008804589505464165

In [273]:
ci0

array([[0.13350769, 0.04004684, 0.14499192, 0.19060605, 0.22783624,
        0.12476813],
       [0.14113332, 0.19598968, 0.16305336, 0.24460255, 0.01669965,
        0.23528007],
       [0.19158664, 0.23112693, 0.23156954, 0.28439579, 0.22270361,
        0.18156047],
       [0.08718169, 0.08487368, 0.00516769, 0.14530301, 0.00578207,
        0.04255375],
       [0.04531161, 0.21573867, 0.13073697, 0.22694107, 0.18254903,
        0.01521281],
       [0.14005073, 0.15994695, 0.17569505, 0.05969608, 0.25997719,
        0.11142458]])

In [274]:
ci1

array([[0.18082025, 0.21059378, 0.09876194, 0.19482774, 0.24737115,
        0.00859667],
       [0.10105422, 0.25109129, 0.09339583, 0.11734425, 0.23848057,
        0.12197454],
       [0.10201867, 0.17062388, 0.12710073, 0.16731576, 0.01581493,
        0.21333788],
       [0.2250545 , 0.26141065, 0.11345579, 0.06748957, 0.26105286,
        0.03143701],
       [0.13663547, 0.00689583, 0.09464234, 0.18208695, 0.01201575,
        0.06479511],
       [0.26693888, 0.16923   , 0.13961166, 0.16288811, 0.24112239,
        0.2055747 ]])